In [ ]:
import numpy as np
import networkx as nx
from random import choice
import matplotlib.pyplot as plt

def compute_efficiency(G):
  '''This function takes a networkx graph and computes its efficiency.'''
  #print(apsp)
  n = G.number_of_nodes()
  if n<2:
    return np.nan
  apsp = list(nx.all_pairs_shortest_path(G))
  pairwise_efficiencies = np.zeros((n,n))
  for i_index, i in enumerate(G.nodes()):
    #print('i', i)
    for tup in apsp:
      if tup[0]==i:
        path_dict = tup[1]
    #path_dict = apsp[i_index][1]
    for j_index, j in enumerate(G.nodes()):
      #print('j',j)
      if i!=j:
        if j in path_dict.keys():
          p = path_dict[j]
          if len(p) > 1:
            pairwise_efficiencies[i_index,j_index] = 1/(len(p)-1)
  efficiency = np.sum(pairwise_efficiencies)/(n*(n-1))
  return efficiency
  #taken from Jessica's notebook 4/4

def degree_distr(x, G):
  array = G.degree
  degree_count = 0
  for node in G.degree:
    if node[1] == x:
      degree_count += 1
  return degree_count / G.number_of_nodes()

def mean_shortest_path(graph):
  if nx.number_of_nodes(graph) == 0:
    return 0
  else:
    n = len(max(nx.connected_components(graph), key=len))
    if n<2:
      return np.nan
    apsp = list(nx.all_pairs_shortest_path(graph))
    shortest_distances = np.zeros((n,n))
    for i_index, i in enumerate(max(nx.connected_components(graph), key=len)):
      #print('i', i)
      for tup in apsp:
        if tup[0]==i:
          path_dict = tup[1]
      #path_dict = apsp[i_index][1]
      for j_index, j in enumerate(max(nx.connected_components(graph), key=len)):
        #print('j',j)
        if i!=j:
          if j in path_dict.keys():
            p = path_dict[j]
            if len(p) > 1:
              shortest_distances[i_index,j_index] = len(p)-1
    mean_shortest = np.sum(shortest_distances) / scipy.special.comb(n, 2)
    return mean_shortest

def computeRobustnessCurve(g, remove_nodes='random', performance='largest_connected_component'):

  # compute the maximum number of nodes to be removed
  n = g.number_of_nodes()

  data_array = np.zeros((2,n), dtype=float) #2 rows of n columns each: top row is number of nodes removed, bottom row is performance measurement
  data_array[0] = np.arange(n)

  if performance=='number_of_nodes':

    def computePerformance(graph):
      output = graph.number_of_nodes()
      return output

  elif performance=='largest_connected_component':

    def computePerformance(graph):
      if(graph.number_of_nodes() == 0):
        return 0
      else:
        nodes_in_cluster = len(max(nx.connected_components(graph), key=len))
        return nodes_in_cluster

  elif performance=='efficiency':

    def computePerformance(graph):
      return compute_efficiency(graph)

  elif performance == "entropy":

    def computePerformance(graph):
      if nx.number_of_nodes(graph) == 0:
        return 0
      else:
        max_degree = sorted(graph.degree, key=lambda x: x[1], reverse=True)[0][1]
        H = 0
        for i in range(max_degree):
          p_k = degree_distr(i, graph)
          H += p_k * np.log(p_k)
        return -H

  elif performance == "mean shortest path":
    def computePerformance(graph):
      return mean_shortest_path(graph)

  elif performance == "average cluster size":
    def computePerformance(graph):
      n = nx.number_of_nodes(graph)
      n_c = nx.connected_components(graph)
      return n/n_c

  elif performance == "relative LCC":

    def computePerformance(graph):
      if(graph.number_of_nodes() == 0):
          return 0
      else:
        nodes_in_cluster = len(max(nx.connected_components(graph), key=len))
        return nodes_in_cluster / graph.number_of_nodes()

  elif performance == "reachability":

    def computePerformance(graph):
      N = graph.number_of_nodes()
      if N==0:
        return 0
      else:
        reach = 0
        # replace
        for i_index, i in enumerate(N, key=len):
          for j_index, j in enumerate(N, key=len):
            if i!=j:
              if nx.has_path(i, j) == True:
                reach += 1

      return .5 * 1/(np.comb(N, 2)) * reach

  elif performance == "transitivity":

    def computePerformance(graph):
      return nx.transitivity(graph)

  elif performance == "resistance distance":
    def computePerformance(graph):
      N = graph.number_of_nodes()
      L = nx.laplacian_matrix(graph)
      L_plus = np.linalg.pinv(L)

      return N * np.trace(L_plus)
  
  elif performance == "natural connectivity":
    def computePerformance(graph):
      N = graph.number_of_nodes()
      return np.log(np.trace(nx.communicability_matrix(graph))) - np.log(N)
      
  else:
    print('Error: I dont know that performance value')
    return 0


  for i in range(n):
    #find a node to remove
    if remove_nodes == 'random':
      v = choice(list(g.nodes()))
    elif remove_nodes == 'attack':
      v = sorted(g.degree, key=lambda x: x[1], reverse=True)[0][0] 
    else:
      print('Error: I dont know that mode of removing nodes')
      v = None #will this error?

    g.remove_node(v)
   
    data_array[1,i] = computePerformance(g)

  return data_array


def plotLineGraphs(data_array): #generate the 8 graphs from 4/4; relate number of nodes, number of edges, 
  plt.figure()

  k = 0
  for i_gt, graph_type in enumerate(graph_types):
    for i_rs, remove_strategy in enumerate(remove_strategies):
      plt.subplot(4,2,1+k)

      for i_nn, number_of_nodes in enumerate(numbers_of_nodes):

        data_for_subplot = data_array[i_gt][i_nn][0][i_rs]

        plt.plot(data_for_subplot[0], np.mean(data_for_subplot[1:], axis=0), label='n='+str(number_of_nodes))

        k += 1

  for i_gt, graph_type in enumerate(graph_types):
    for i_rs, remove_strategy in enumerate(remove_strategies):
      plt.subplot(4,2,1+k)

      for i_ne, number_of_edges in enumerate(numbers_of_edges):

        data_for_subplot = data_array[i_gt][0][i_ne][i_rs]

        plt.plot(data_for_subplot[0], np.mean(data_for_subplot[1:], axis=0), label='m='+str(number_of_edges))

        k += 1

    

  
  #for i in range(len(data_array)-1):
  #    plt.plot(data_array[0], data_array[i+1]) #index is i+1 because first row is number of nodes removed, so you have to skip by adding one.

  ##plt.subplot(1, 2, 2)
  #for i in range(len(data_array)-1):
  #  plt.plot(data_array[i+1], data_array[0])


def computeRobustnessCurves(number_of_nodes=100, number_of_edges=20, graph_type='ER', remove_nodes='random', performance='largest_connected_component', num_trials=10):

  data_array = np.zeros((num_trials+1,number_of_nodes), dtype=float)
  data_array[0] = np.arange(number_of_nodes)

  for i in range(num_trials):
    g = construct_a_network(number_of_nodes=number_of_nodes, number_of_edges=number_of_edges, graph_type=graph_type)
    data = computeRobustnessCurve(g, remove_nodes=remove_nodes, performance=performance)
    data_array[i+1] = data[1]

  return data_array


def completeRobustnessData(graph_types=['ER', 'BA'], 
                           numbers_of_nodes=np.arange(20,101,20), 
                           numbers_of_edges=[1,2,3], 
                           remove_strategies = ['random', 'attack'], 
                           performance='efficiency'):
  # initialize some big list
  LIST = [[[[0 for i in range(len(remove_strategies))] 
            for j in range(len(numbers_of_edges))] 
           for k in range(len(numbers_of_nodes))] 
          for l in range(len(graph_types))]

  #LIST = []
  #for l in range(len(graph_types)):
  #  LIST.append([])
  #  for k in range(len(numbers_of_nodes)):
  #    LIST[-1].append([])

  for i_gt, graph_type in enumerate(graph_types):
    for i_nn, number_of_nodes in enumerate(numbers_of_nodes):
      for i_ne, number_of_edges in enumerate(numbers_of_edges):
        for i_rs, remove_strategy in enumerate(remove_strategies):

          data = computeRobustnessCurves(graph_type=graph_type, ..., performance=performance) #TODO
          LIST[i_gt][i_nn][i_ne][i_rs] = data
  return LIST

SyntaxError: ignored

In [ ]:
elif performance == "resistance distance":
  def computePerformance(graph):
    N = graph.number_of_nodes()
    L = nx.laplacian_matrix(graph)
    L_plus = np.linalg.pinv(L)

    return N * np.(L_plus)


In [ ]:
# construct a network
def construct_a_network(number_of_nodes, number_of_edges, graph_type): #n = #nodes, m = #edges
  if graph_type == 'ER':
    p = 2*number_of_edges*(number_of_nodes-number_of_edges)/(number_of_nodes*(number_of_nodes-1))
    g = nx.erdos_renyi_graph(number_of_nodes, p, seed=None, directed=False)
    return g
  elif graph_type == 'SF':
    g2 = nx.barabasi_albert_graph(number_of_nodes, number_of_edges)
    return g2
  else:
    print("Error: invalid graph_type")
#g = construct_a_network(number_of_nodes=100, number_of_edges=20, graph_type='ER')

# get robustness data
#data = compute_robustness_curve(g, remove_nodes='random', performance='largest_connected_component')

data = computeRobustnessCurves(number_of_nodes=100, number_of_edges=2, graph_type='ER', # parameters for graph construction
                        remove_nodes='random', performance='efficiency' #parameter for robustness computation
                        )

# plot data
plt.plot(data[0], data[1]) 

#instead, graph over the 3 attributes–graph type, removal, axis via for loops

NameError: ignored

In [ ]:
DATA = completeRobustnessData(graph_types=['ER', 'BA'], 
                           numbers_of_nodes=np.arange(20,101,20), 
                           numbers_of_edges=[1,2,3], 
                           remove_strategies = ['random', 'attack'], 
                           performance='efficiency')

plotLineGraphs(DATA, graph_types=['ER', 'BA'], 
                           numbers_of_nodes=np.arange(20,101,20), 
                           numbers_of_edges=[1,2,3], 
                           remove_strategies = ['random', 'attack'], 
                           performance='efficiency') #this one generates 10 layers

[[0, 0], [0, 0], [0, 0], [0, 0]]

In [ ]:
def my_function(number1=1, number2=1, optional_string='default'):
  for i in range(number2):
    print(number1)
  print(optional_string)

In [ ]:
my_function()

1
default


In [ ]:
#for next time: code efficiency measurements: 1 from each subsection in 6 (i.e., 6.2-6.6, 5 total)
#next time: how to save data, run simulations

In [ ]:
# def completeRobustnessData(graph_types=['ER', 'SF'], 
#                            numbers_of_nodes=np.arange(5, 21, 5), #20,101,20: i.e., 20, 40 ,60, 80, 100 nodes 
#                            numbers_of_edges=[1,2,3], 
#                            remove_strategies = ['random', 'attack'], 
#                            performance='efficiency'):

def plot_graphs(graph_types=['ER', 'SF'], 
                           numbers_of_nodes=np.arange(20, 101, 20), #20,101,20: i.e., 20, 40 ,60, 80, 100 nodes 
                           numbers_of_edges=[1,2,3], 
                           remove_strategies = ['random', 'attack'], 
                           performance='efficiency', to_vary = 'nodes', vary_index = 1): 
  LIST = completeRobustnessData(graph_types, numbers_of_nodes, numbers_of_edges, remove_strategies, performance)

  if to_vary == 'nodes':
    # start plotting
    fig = plt.figure()

    # 1. plot performance as function of n, 2 * 2 * 5 figures
    x = 1
    while x < 5:
      for i_gt, graph_type in enumerate(graph_types):
        for i_rs, remove_strategy in enumerate(remove_strategies):
          ax1 = plt.subplot(2,2,x) #1 subplot for every combination of graph type/rs
          x +=1
          print(x)
          for i_nn, number_of_node in enumerate(numbers_of_nodes):
                # print(range(numbers_of_nodes[i_nn]))
                # print(LIST[i_gt][i_nn][i_ne][i_rs])
                ax1.plot(range(numbers_of_nodes[i_nn]), np.mean(LIST[i_gt][i_nn][vary_index][i_rs][1:], axis = 0), 'o-', label = "number of nodes="+str(numbers_of_nodes[i_nn]))
                ax1.set_title(str(performance)+str(graph_type)+" "+str(remove_strategy)+": n")
                ax1.legend()
                ax1.set_xlabel('n (number nodes removed)')
                ax1.set_ylabel(performance)
                #plt.show
          # fig.tight_layout()
      plt.subplots_adjust(left=None, bottom=None, right=2, top=2, wspace=None, hspace=None)
      plt.show()

  elif to_vary == 'edges':
    # start plotting
    fig = plt.figure()

    # 1. plot performance as function of n, 2 * 2 * 5 figures
    x = 1
    while x < 5:
      for i_gt, graph_type in enumerate(graph_types):
        for i_rs, remove_strategy in enumerate(remove_strategies):
          ax1 = plt.subplot(2,2,x) #1 subplot for every combination of graph type/rs
          x +=1
          print(x)
          for i_ne, number_of_edge in enumerate(numbers_of_edges):
                # print(range(numbers_of_nodes[i_nn]))
                # print(LIST[i_gt][i_nn][i_ne][i_rs])
                ax1.plot(range(numbers_of_nodes[1]), np.mean(LIST[i_gt][vary_index][i_ne][i_rs][1:], axis = 0), 'o-', label = "number of edges="+str(numbers_of_edges[i_ne]))
                ax1.set_title(str(performance)+str(graph_type)+" "+str(remove_strategy)+": n")
                ax1.legend()
                ax1.set_xlabel('n (number nodes removed)')
                ax1.set_ylabel(performance)
                #plt.show
          # fig.tight_layout()
      plt.subplots_adjust(left=None, bottom=None, right=2, top=2, wspace=None, hspace=None)
      plt.show()
    else:
      print("Please vary either nodes or edges.")

In [ ]:
plot_graphs(['ER', 'SF'], np.arange(20, 101, 20), [1,2,3], ['random', 'attack'], 'efficiency')

In [ ]:
def riemann_percent(array):
  biggest = array[0]
  return np.sum(array) * (1 / array.size) / biggest